In [108]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [109]:
data = pd.read_csv('ListingSecurityList.csv', error_bad_lines=False, sep = ';')

In [110]:
eqs = data[data.s_sec_type_name_dop == 'Акции']

In [111]:
# depricated: financе.google.com
# start_url = 'https://finance.google.com/finance?q='
# url = start_url + 'AA'
# html = urlopen(url)
# bsObj = BeautifulSoup(html.read(), "lxml")
# bsObj.find('a',{'id':'sector'})

In [124]:
url = start_url + 'AA'
html = urlopen(url)
bsObj = BeautifulSoup(html.read(), "lxml")

# print(bsObj.prettify())

In [115]:
for i, n in enumerate(bsObj.findAll("td")):
    print(i)
    print(n.text)

0
Previous Close
1
19.05
2
Open
3
19.10
4
Bid
5
19.42 x 1100
6
Ask
7
19.43 x 1400
8
Day's Range
9
18.94 - 19.49
10
52 Week Range
11
16.46 - 41.01
12
Volume
13
2,719,455
14
Avg. Volume
15
3,599,346
16
Market Cap
17
3.656B
18
Beta (3Y Monthly)
19
1.65
20
PE Ratio (TTM)
21
N/A
22
EPS (TTM)
23
-3.12
24
Earnings Date
25
Oct 16, 2019
26
Forward Dividend & Yield
27
N/A (N/A)
28
Ex-Dividend Date
29
N/A
30
1y Target Est
31
26.83


In [116]:
#depricated
def get_data_google(url):
    result = {}
    html = urlopen(url)
    bsObj = BeautifulSoup(html.read(), "lxml")
    for snap in bsObj.find_all("table", {"class":"snap-data"}):
        for trs in snap.find_all("tr"):
            name = trs.find('td',{'class':'key'}).text
            name = re.sub('\n','',name)
            value = trs.find('td',{'class':'val'}).text
            value = re.sub('\n','',value)
            result[name] = value
        result['price'] = re.sub('\n','',bsObj.find('span',{'class':'pr'}).text)
        if bsObj.find('a',{'id':'sector'}):
            sector, industry = bsObj.find('div',{'class':'g-section g-tpl-50-50 g-split'}).div.text.split('>')
            result['sector'] = re.sub('\n','',sector)
            result['industry'] = re.sub('\n','',industry)
    return result

In [117]:
def get_data(url):
    result = {}
    html = urlopen(url)
    bsObj = BeautifulSoup(html.read(), "lxml")
    for i in range(0,32,2):
#         print(i)
#         print(bsObj.findAll("td")[i].text)
#         print(bsObj.findAll("td")[i+1].text)
        result[bsObj.findAll("td")[i].text] = bsObj.findAll("td")[i+1].text
    return result

In [118]:
get_data(start_url + 'AA')

{'1y Target Est': '26.83',
 '52 Week Range': '16.46 - 41.01',
 'Ask': '19.43 x 1400',
 'Avg. Volume': '3,599,346',
 'Beta (3Y Monthly)': '1.65',
 'Bid': '19.42 x 1100',
 "Day's Range": '18.94 - 19.49',
 'EPS (TTM)': '-3.12',
 'Earnings Date': 'Oct 16, 2019',
 'Ex-Dividend Date': 'N/A',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Market Cap': '3.656B',
 'Open': '19.10',
 'PE Ratio (TTM)': 'N/A',
 'Previous Close': '19.05',
 'Volume': '2,719,455'}

In [123]:
# start_url = 'https://finance.google.com/finance?q='
start_url = 'https://finance.yahoo.com/quote/'

for s in eqs.s_RTS_code:
    print(s)
    url = start_url + s
    try:
        market_data = get_data(url)
        print(market_data)
        for key,value in market_data.items():
            if key not in list(data.columns.values):
                data[key] = 0
            data.loc[data.s_RTS_code == s, key] = value
    except:
        print('bad')

A
{'Previous Close': '73.87', 'Open': '74.08', 'Bid': '75.77 x 800', 'Ask': '75.70 x 800', "Day's Range": '74.06 - 75.91', '52 Week Range': '61.01 - 82.27', 'Volume': '1,218,771', 'Avg. Volume': '2,663,055', 'Market Cap': '23.317B', 'Beta (3Y Monthly)': '1.28', 'PE Ratio (TTM)': '22.70', 'EPS (TTM)': '3.34', 'Earnings Date': 'Nov 18, 2019 - Nov 22, 2019', 'Forward Dividend & Yield': '0.66 (0.89%)', 'Ex-Dividend Date': '2019-07-01', '1y Target Est': '82.58'}
AA
{'Previous Close': '19.05', 'Open': '19.10', 'Bid': '19.42 x 1100', 'Ask': '19.43 x 1400', "Day's Range": '18.94 - 19.49', '52 Week Range': '16.46 - 41.01', 'Volume': '2,719,455', 'Avg. Volume': '3,599,346', 'Market Cap': '3.656B', 'Beta (3Y Monthly)': '1.65', 'PE Ratio (TTM)': 'N/A', 'EPS (TTM)': '-3.12', 'Earnings Date': 'Oct 16, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': 'N/A', '1y Target Est': '26.83'}
AABA
{'Previous Close': '19.63', 'Open': '19.56', 'Bid': '19.61 x 1000', 'Ask': '19.62 x 2200', "Day

{'Previous Close': 'N/A', 'YTD Return': 'N/A', 'Expense Ratio (net)': 'N/A', 'Category': 'N/A', 'Last Cap Gain': 'N/A', 'Morningstar Rating': 'N/A', 'Morningstar Risk Rating': 'N/A', 'Sustainability Rating': 'N/A', 'Net Assets': 'N/A', 'Beta (3Y Monthly)': 'N/A', 'Yield': 'N/A', '5y Average Return': 'N/A', 'Holdings Turnover': 'N/A', 'Last Dividend': 'N/A', 'Average for Category': 'N/A', 'Inception Date': 'N/A'}
AFL
{'Previous Close': '51.18', 'Open': '51.21', 'Bid': '49.00 x 800', 'Ask': '53.87 x 1300', "Day's Range": '51.21 - 52.29', '52 Week Range': '41.45 - 57.18', 'Volume': '3,099,573', 'Avg. Volume': '3,431,284', 'Market Cap': '38.122B', 'Beta (3Y Monthly)': '0.50', 'PE Ratio (TTM)': '12.70', 'EPS (TTM)': '4.10', 'Earnings Date': 'Oct 24, 2019', 'Forward Dividend & Yield': '1.08 (2.14%)', 'Ex-Dividend Date': '2019-08-20', '1y Target Est': '52.07'}
AGN
{'Previous Close': '167.22', 'Open': '167.30', 'Bid': '168.37 x 800', 'Ask': '168.34 x 800', "Day's Range": '167.10 - 168.60', '52

{'Previous Close': '135.17', 'Open': '135.44', 'Bid': '135.95 x 900', 'Ask': '135.99 x 800', "Day's Range": '134.84 - 136.62', '52 Week Range': '95.69 - 153.91', 'Volume': '889,475', 'Avg. Volume': '1,045,542', 'Market Cap': '17.846B', 'Beta (3Y Monthly)': '1.99', 'PE Ratio (TTM)': '10.03', 'EPS (TTM)': '13.58', 'Earnings Date': 'Oct 21, 2019 - Oct 25, 2019', 'Forward Dividend & Yield': '3.88 (2.87%)', 'Ex-Dividend Date': '2019-08-02', '1y Target Est': '166.20'}
AMT
{'Previous Close': '223.90', 'Open': '223.97', 'Bid': '225.55 x 800', 'Ask': '226.14 x 900', "Day's Range": '223.66 - 227.40', '52 Week Range': '140.40 - 242.00', 'Volume': '1,538,024', 'Avg. Volume': '1,725,288', 'Market Cap': '98.269B', 'Beta (3Y Monthly)': '-0.01', 'PE Ratio (TTM)': '68.26', 'EPS (TTM)': '3.31', 'Earnings Date': 'Oct 31, 2019', 'Forward Dividend & Yield': '3.61 (1.64%)', 'Ex-Dividend Date': '2019-06-18', '1y Target Est': '221.53'}
AMZN
{'Previous Close': '1,724.42', 'Open': '1,726.02', 'Bid': '1,741.05 x

{'Previous Close': '4.3200', 'Open': '4.3600', 'Bid': '4.4900 x 3000', 'Ask': '4.5500 x 1300', "Day's Range": '4.3000 - 4.5100', '52 Week Range': '1.3000 - 4.8800', 'Volume': '3,743,939', 'Avg. Volume': '7,821,007', 'Market Cap': '1.949B', 'Beta (3Y Monthly)': '1.30', 'PE Ratio (TTM)': 'N/A', 'EPS (TTM)': '-0.0920', 'Earnings Date': 'Oct 31, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': '2015-11-12', '1y Target Est': '3.94'}
AVY
{'Previous Close': '109.45', 'Open': '109.80', 'Bid': '111.48 x 800', 'Ask': '111.50 x 800', "Day's Range": '109.78 - 111.83', '52 Week Range': '82.89 - 120.65', 'Volume': '463,542', 'Avg. Volume': '492,295', 'Market Cap': '9.372B', 'Beta (3Y Monthly)': '1.46', 'PE Ratio (TTM)': '39.34', 'EPS (TTM)': '2.83', 'Earnings Date': 'Oct 21, 2019 - Oct 25, 2019', 'Forward Dividend & Yield': '2.32 (2.12%)', 'Ex-Dividend Date': '2019-09-03', '1y Target Est': '123.11'}
AXP
{'Previous Close': '112.55', 'Open': '112.66', 'Bid': '114.37 x 800', 'Ask': '

{'Previous Close': '42.43', 'Open': '42.59', 'Bid': '42.01 x 1000', 'Ask': '45.44 x 900', "Day's Range": '42.14 - 42.77', '52 Week Range': '40.52 - 54.27', 'Volume': '10,211,958', 'Avg. Volume': '4,792,273', 'Market Cap': '40.243B', 'Beta (3Y Monthly)': '1.04', 'PE Ratio (TTM)': '11.08', 'EPS (TTM)': '3.85', 'Earnings Date': 'Oct 16, 2019', 'Forward Dividend & Yield': '1.24 (2.92%)', 'Ex-Dividend Date': '2019-07-26', '1y Target Est': '47.42'}
BLK
{'Previous Close': '419.61', 'Open': '421.15', 'Bid': '410.00 x 1000', 'Ask': '456.00 x 800', "Day's Range": '419.09 - 427.70', '52 Week Range': '360.79 - 487.45', 'Volume': '342,998', 'Avg. Volume': '591,315', 'Market Cap': '66.603B', 'Beta (3Y Monthly)': '1.45', 'PE Ratio (TTM)': '16.25', 'EPS (TTM)': '26.30', 'Earnings Date': 'Oct 15, 2019', 'Forward Dividend & Yield': '13.20 (3.14%)', 'Ex-Dividend Date': '2019-09-04', '1y Target Est': '517.92'}
BLL
{'Previous Close': '71.64', 'Open': '71.92', 'Bid': '70.11 x 800', 'Ask': '74.01 x 800', "Da

...

{'Previous Close': '61.36', 'Open': '61.44', 'Bid': '62.09 x 1000', 'Ask': '62.05 x 1300', "Day's Range": '61.21 - 62.18', '52 Week Range': '42.50 - 62.36', 'Volume': '3,142,479', 'Avg. Volume': '4,750,014', 'Market Cap': '64.634B', 'Beta (3Y Monthly)': '0.26', 'PE Ratio (TTM)': '14.61', 'EPS (TTM)': '4.25', 'Earnings Date': 'Oct 30, 2019', 'Forward Dividend & Yield': '2.48 (4.06%)', 'Ex-Dividend Date': '2019-08-16', '1y Target Est': '58.20'}
SOHU
{'Previous Close': '10.80', 'Open': '10.76', 'Bid': '10.80 x 1300', 'Ask': '10.93 x 1300', "Day's Range": '10.60 - 10.92', '52 Week Range': '8.79 - 23.60', 'Volume': '162,229', 'Avg. Volume': '452,661', 'Market Cap': '427.591M', 'Beta (3Y Monthly)': '1.72', 'PE Ratio (TTM)': 'N/A', 'EPS (TTM)': '-3.31', 'Earnings Date': 'Nov 4, 2019 - Nov 8, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': 'N/A', '1y Target Est': '18.84'}
SPG
{'Previous Close': '147.50', 'Open': '148.04', 'Bid': '148.25 x 1100', 'Ask': '148.02 x 800', "Day'

{'Previous Close': '56.62', 'Open': '56.61', 'Bid': '54.44 x 800', 'Ask': '56.59 x 800', "Day's Range": '55.65 - 56.88', '52 Week Range': '49.92 - 67.62', 'Volume': '1,224,654', 'Avg. Volume': '1,674,909', 'Market Cap': '12.287B', 'Beta (3Y Monthly)': '0.99', 'PE Ratio (TTM)': '13.71', 'EPS (TTM)': '4.13', 'Earnings Date': 'Oct 30, 2019', 'Forward Dividend & Yield': '2.28 (4.02%)', 'Ex-Dividend Date': '2019-08-29', '1y Target Est': '58.89'}
TDC
{'Previous Close': '29.61', 'Open': '29.79', 'Bid': '29.83 x 800', 'Ask': '29.84 x 900', "Day's Range": '29.68 - 30.14', '52 Week Range': '29.00 - 49.42', 'Volume': '790,877', 'Avg. Volume': '1,030,974', 'Market Cap': '3.419B', 'Beta (3Y Monthly)': '1.24', 'PE Ratio (TTM)': '159.73', 'EPS (TTM)': '0.19', 'Earnings Date': 'Oct 30, 2019 - Nov 4, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': 'N/A', '1y Target Est': '44.12'}
TDG
{'Previous Close': '507.23', 'Open': '509.28', 'Bid': '511.99 x 1000', 'Ask': '513.57 x 900', "Day's

{'Previous Close': '40.00', 'Open': '40.36', 'Bid': '40.35 x 800', 'Ask': '40.36 x 1400', "Day's Range": '39.83 - 40.59', '52 Week Range': '26.19 - 45.86', 'Volume': '6,230,963', 'Avg. Volume': '13,016,822', 'Market Cap': '30.965B', 'Beta (3Y Monthly)': '0.23', 'PE Ratio (TTM)': '13.34', 'EPS (TTM)': '3.03', 'Earnings Date': 'Oct 24, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': 'N/A', '1y Target Est': '42.12'}
TWX
{'Previous Close': 'N/A', 'YTD Return': 'N/A', 'Expense Ratio (net)': 'N/A', 'Category': 'N/A', 'Last Cap Gain': 'N/A', 'Morningstar Rating': 'N/A', 'Morningstar Risk Rating': 'N/A', 'Sustainability Rating': 'N/A', 'Net Assets': 'N/A', 'Beta (3Y Monthly)': 'N/A', 'Yield': 'N/A', '5y Average Return': 'N/A', 'Holdings Turnover': 'N/A', 'Last Dividend': 'N/A', 'Average for Category': 'N/A', 'Inception Date': 'N/A'}
TXN
{'Previous Close': '128.70', 'Open': '129.31', 'Bid': '130.15 x 800', 'Ask': '130.21 x 1200', "Day's Range": '129.02 - 130.31', '52 Week Ra

{'Previous Close': '114.79', 'Open': '115.16', 'Bid': '113.87 x 800', 'Ask': '113.53 x 800', "Day's Range": '112.63 - 115.66', '52 Week Range': '101.42 - 142.50', 'Volume': '548,879', 'Avg. Volume': '730,147', 'Market Cap': '10.201B', 'Beta (3Y Monthly)': '0.93', 'PE Ratio (TTM)': '30.97', 'EPS (TTM)': '3.67', 'Earnings Date': 'Oct 23, 2019', 'Forward Dividend & Yield': 'N/A (N/A)', 'Ex-Dividend Date': '1998-12-02', '1y Target Est': '140.36'}
VFC
{'Previous Close': '87.17', 'Open': '87.42', 'Bid': '88.54 x 800', 'Ask': '88.86 x 1100', "Day's Range": '87.39 - 88.94', '52 Week Range': '63.26 - 91.54', 'Volume': '1,795,308', 'Avg. Volume': '1,667,265', 'Market Cap': '35.128B', 'Beta (3Y Monthly)': '1.44', 'PE Ratio (TTM)': '30.99', 'EPS (TTM)': '2.86', 'Earnings Date': 'Oct 25, 2019', 'Forward Dividend & Yield': '1.72 (1.98%)', 'Ex-Dividend Date': '2019-09-09', '1y Target Est': '100.38'}
VIAB
{'Previous Close': '22.93', 'Open': '22.98', 'Bid': '23.24 x 4000', 'Ask': '23.26 x 2200', "Day's

bad
WMB
{'Previous Close': '23.56', 'Open': '23.64', 'Bid': '23.50 x 1800', 'Ask': '23.69 x 1800', "Day's Range": '23.48 - 23.77', '52 Week Range': '20.36 - 29.55', 'Volume': '5,960,920', 'Avg. Volume': '7,982,711', 'Market Cap': '28.433B', 'Beta (3Y Monthly)': '1.43', 'PE Ratio (TTM)': '537.50', 'EPS (TTM)': '0.04', 'Earnings Date': 'Oct 30, 2019', 'Forward Dividend & Yield': '1.52 (6.50%)', 'Ex-Dividend Date': '2019-09-12', '1y Target Est': '30.00'}
WMT
{'Previous Close': '116.31', 'Open': '116.40', 'Bid': '118.02 x 900', 'Ask': '118.21 x 900', "Day's Range": '116.19 - 118.30', '52 Week Range': '85.78 - 119.86', 'Volume': '4,220,230', 'Avg. Volume': '5,603,803', 'Market Cap': '335.532B', 'Beta (3Y Monthly)': '0.65', 'PE Ratio (TTM)': '26.72', 'EPS (TTM)': '4.42', 'Earnings Date': 'Nov 14, 2019', 'Forward Dividend & Yield': '2.12 (1.83%)', 'Ex-Dividend Date': '2019-12-05', '1y Target Est': '119.57'}
WRK
bad
WU
bad
WY
{'Previous Close': '26.94', 'Open': '26.94', 'Bid': '27.01 x 1800', 

In [129]:
data.head()

,Row,s_RTS_code,e_full_name,s_sec_type_name_dop,s_sec_form_name_full,s_zagolovok_adr,s_ISIN_code,s_CFI_code,s_trade_lot,s_step_price,...,RH,170.23,VHT,167.62,ETF,AHT,3.15,RHI,54.09,HT
0,1,A,"Agilent Technologies, Inc.",Акции,Акции иностранного эмитента обыкновенные,NaN,US00846U1016,ESVUFN,"1,00","0,01",...,0,0,0,0,0,0,0,0,0,0
1,2,AA,Alcoa Corporation,Акции,Акции иностранного эмитента обыкновенные,NaN,US0138721065,ESVUFR,"1,00","0,01",...,0,0,0,0,0,0,0,0,0,0
2,3,AABA,Altaba Inc.,Акции,Акции иностранного эмитента обыкновенные,NaN,US0213461017,ESVUFR,"1,00","0,01",...,0,0,0,0,0,0,0,0,0,0
3,4,AAL,American Airlines Group Inc.,Акции,Акции иностранного эмитента обыкновенные,NaN,US02376R1023,ESVUFR,"1,00","0,01",...,0,0,0,0,0,0,0,0,0,0
4,5,AAP,"Advance Auto Parts, Inc.",Акции,Акции иностранного эмитента обыкновенные,NaN,US00751Y1064,ESVUFR,"1,00","0,01",...,0,0,0,0,0,0,0,0,0,0


In [127]:
for name in data.columns[12:26]:
    data[name] = data[name].str.replace('.',',')

In [128]:
data.to_csv('result_w_prices20191006.csv', sep=';')